In [2]:
import glob 
import pandas as pd
import numpy as np
import requests
import time
import os
import glob
from datetime import date, timedelta
from pathlib import Path

dir_path = '../data/vehicles/*.parquet'
file_names = sorted(glob.glob(dir_path))

df = []
for file in file_names:
        df.append(pd.read_parquet(file))
        print(file)
df = pd.concat(df)

../data/vehicles/vehicle_count_hourly_2020-01-01.parquet
../data/vehicles/vehicle_count_hourly_2020-01-02.parquet
../data/vehicles/vehicle_count_hourly_2020-01-03.parquet
../data/vehicles/vehicle_count_hourly_2020-01-04.parquet
../data/vehicles/vehicle_count_hourly_2020-01-05.parquet
../data/vehicles/vehicle_count_hourly_2020-01-06.parquet
../data/vehicles/vehicle_count_hourly_2020-01-07.parquet
../data/vehicles/vehicle_count_hourly_2020-01-08.parquet
../data/vehicles/vehicle_count_hourly_2020-01-09.parquet
../data/vehicles/vehicle_count_hourly_2020-01-10.parquet
../data/vehicles/vehicle_count_hourly_2020-01-11.parquet
../data/vehicles/vehicle_count_hourly_2020-01-12.parquet
../data/vehicles/vehicle_count_hourly_2020-01-13.parquet
../data/vehicles/vehicle_count_hourly_2020-01-14.parquet
../data/vehicles/vehicle_count_hourly_2020-01-15.parquet
../data/vehicles/vehicle_count_hourly_2020-01-16.parquet
../data/vehicles/vehicle_count_hourly_2020-01-17.parquet
../data/vehicles/vehicle_count_

In [7]:
df

,id_site,t,sens,nb_usagers,label,trajectoire,voie,id_trajectoire,coordonnees_geo,mode
0,10004,2020-05-22T22:00:00+00:00,O-E,4,[Paris] Rivoli x Nicolas Flamel,3 -> 2,Coronapiste,10004_3 -> 2,"[48.858273, 2.349109]",Trottinettes + vélos
1,10004,2020-05-22T23:00:00+00:00,O-E,3,[Paris] Rivoli x Nicolas Flamel,3 -> 2,Coronapiste,10004_3 -> 2,"[48.858273, 2.349109]",Trottinettes + vélos
2,10004,2020-05-22T22:00:00+00:00,E-O,5,[Paris] Rivoli x Nicolas Flamel,5 -> 3,Voie de circulation générale,10004_5 -> 3,"[48.858273, 2.349109]",Autobus et autocars
3,10004,2020-05-22T23:00:00+00:00,E-O,1,[Paris] Rivoli x Nicolas Flamel,2 -> 1,Piste cyclable,10004_2 -> 1,"[48.858273, 2.349109]","Véhicules légers < 3,5t"
4,10004,2020-05-22T23:00:00+00:00,E-O,85,[Paris] Rivoli x Nicolas Flamel,5 -> 3,Voie de circulation générale,10004_5 -> 3,"[48.858273, 2.349109]","Véhicules légers < 3,5t"
...,...,...,...,...,...,...,...,...,...,...
1478,10015,2023-05-21T12:00:00+00:00,N-S,57,[Paris] Amsterdam x Clichy,1 -> 1,Voie de circulation générale,10015_1 -> 1,"[48.883513, 2.327263]","Véhicules légers < 3,5t"
1479,10015,2023-05-21T14:00:00+00:00,N-S,23,[Paris] Amsterdam x Clichy,1 -> 1,Voie de circulation générale,10015_1 -> 1,"[48.883513, 2.327263]",2 roues motorisées
1480,10015,2023-05-21T18:00:00+00:00,S-N,93,[Paris] Amsterdam x Clichy,2 -> 2,Coronapiste,10015_2 -> 2,"[48.883513, 2.327263]",Trottinettes + vélos
1481,10019,2023-05-21T00:00:00+00:00,SO-NE,1,[Paris] Quai de Valmy,1 -> 1,Voie de circulation générale,10019_1 -> 1,"[48.877338, 2.365016]",2 roues motorisées


In [9]:
df['label'].unique()

array(['[Paris] Rivoli x Nicolas Flamel', '[Paris] Amsterdam x Clichy',
       '[Paris] Quai de Valmy', '[Paris] Quai de Jemmapes',
       '[Paris] CF892 Rivoli x Bourdonnais',
       '[Paris] CF318 Rivoli x Lobau',
       '[Paris] CF4 Poissonnière x Montmartre OUEST',
       '[Paris] CF4 Poissonnière x Montmartre EST',
       '[Paris] CF1 Rivoli x Sébastopol', None], dtype=object)

In [10]:

## junctions in the city of paris
junction_labels = ['[Paris] Rivoli x Nicolas Flamel',
                    '[Paris] Amsterdam x Clichy',
                    '[Paris] Quai de Valmy',
                    '[Paris] Quai de Jemmapes',
                    '[Paris] CF892 Rivoli x Bourdonnais',
                    '[Paris] CF318 Rivoli x Lobau',
                    '[Paris] CF1 Rivoli x Sébastopol',
                    '[Paris] CF4 Poissonnière x Montmartre EST',
                    '[Paris] CF4 Poissonnière x Montmartre OUEST']


In [11]:
junction = junction_labels[0]

In [12]:
junction

'[Paris] Rivoli x Nicolas Flamel'

In [13]:
        df_junction = df[df['label'] == junction].reset_index(drop=True)
        df_junction['latitude'] = df_junction['coordonnees_geo'].apply(lambda x : x[0])
        df_junction['longitude'] = df_junction['coordonnees_geo'].apply(lambda x : x[1])
        df_junction = df_junction.sort_values(by='t').reset_index(drop=True)


In [17]:
df_junction.head()

,id_site,t,sens,nb_usagers,label,trajectoire,voie,id_trajectoire,coordonnees_geo,mode,latitude,longitude
0,10004,2020-05-22T22:00:00+00:00,O-E,4,[Paris] Rivoli x Nicolas Flamel,3 -> 2,Coronapiste,10004_3 -> 2,"[48.858273, 2.349109]",Trottinettes + vélos,48.858273,2.349109
1,10004,2020-05-22T22:00:00+00:00,O-E,95,[Paris] Rivoli x Nicolas Flamel,1 -> 1,Piste cyclable,10004_1 -> 1,"[48.858273, 2.349109]",Trottinettes + vélos,48.858273,2.349109
2,10004,2020-05-22T22:00:00+00:00,E-O,5,[Paris] Rivoli x Nicolas Flamel,5 -> 3,Voie de circulation générale,10004_5 -> 3,"[48.858273, 2.349109]",Autobus et autocars,48.858273,2.349109
3,10004,2020-05-22T22:00:00+00:00,E-O,31,[Paris] Rivoli x Nicolas Flamel,4 -> 2,Coronapiste,10004_4 -> 2,"[48.858273, 2.349109]",Trottinettes + vélos,48.858273,2.349109
4,10004,2020-05-22T22:00:00+00:00,E-O,85,[Paris] Rivoli x Nicolas Flamel,5 -> 3,Voie de circulation générale,10004_5 -> 3,"[48.858273, 2.349109]","Véhicules légers < 3,5t",48.858273,2.349109


In [19]:
df_junction.groupby(['label','latitude','longitude','t','mode'])['nb_usagers'].sum().reset_index()

,label,latitude,longitude,t,mode,nb_usagers
0,[Paris] Rivoli x Nicolas Flamel,48.858273,2.349109,2020-05-22T22:00:00+00:00,Autobus et autocars,5
1,[Paris] Rivoli x Nicolas Flamel,48.858273,2.349109,2020-05-22T22:00:00+00:00,Trottinettes + vélos,191
2,[Paris] Rivoli x Nicolas Flamel,48.858273,2.349109,2020-05-22T22:00:00+00:00,"Véhicules légers < 3,5t",85
3,[Paris] Rivoli x Nicolas Flamel,48.858273,2.349109,2020-05-22T23:00:00+00:00,Trottinettes + vélos,118
4,[Paris] Rivoli x Nicolas Flamel,48.858273,2.349109,2020-05-22T23:00:00+00:00,"Véhicules légers < 3,5t",86
...,...,...,...,...,...,...
139681,[Paris] Rivoli x Nicolas Flamel,48.858273,2.349109,2023-05-21T21:00:00+00:00,Autobus et autocars,4
139682,[Paris] Rivoli x Nicolas Flamel,48.858273,2.349109,2023-05-21T21:00:00+00:00,Trottinettes,51
139683,[Paris] Rivoli x Nicolas Flamel,48.858273,2.349109,2023-05-21T21:00:00+00:00,"Véhicules lourds > 3,5t",4
139684,[Paris] Rivoli x Nicolas Flamel,48.858273,2.349109,2023-05-21T21:00:00+00:00,"Véhicules légers < 3,5t",152


In [20]:
        df_junction = df_junction.groupby(['label','latitude','longitude','t','mode'])['nb_usagers'].sum().reset_index()
        df_junction = df_junction.groupby(['label','latitude','longitude','t'])[['mode','nb_usagers']].agg(list).reset_index()

        df_junction['mode_&_nb_usagers'] = [list(x) for x in map(zip, df_junction['mode'], df_junction['nb_usagers'])]
        df_junction['mode_&_nb_usagers'] = df_junction['mode_&_nb_usagers'] .apply(lambda x : dict(x))


        df_junction['2 roues motorisées'] = df_junction['mode_&_nb_usagers'].apply(lambda x : x['2 roues motorisées'] if '2 roues motorisées' in x.keys() else np.nan)
        df_junction['Trottinettes'] = df_junction['mode_&_nb_usagers'].apply(lambda x : x['Trottinettes'] if 'Trottinettes' in x.keys() else np.nan)
        df_junction['Véhicules légers < 3,5t'] = df_junction['mode_&_nb_usagers'].apply(lambda x : x['Véhicules légers < 3,5t'] if 'Véhicules légers < 3,5t' in x.keys() else np.nan)
        df_junction['Vélos'] = df_junction['mode_&_nb_usagers'].apply(lambda x : x['Vélos'] if 'Vélos' in x.keys() else np.nan)
        df_junction['Autobus et autocars'] = df_junction['mode_&_nb_usagers'].apply(lambda x : x['Autobus et autocars'] if 'Autobus et autocars' in x.keys() else np.nan)
        df_junction['Véhicules lourds > 3,5t'] = df_junction['mode_&_nb_usagers'].apply(lambda x : x['Véhicules lourds > 3,5t'] if 'Véhicules lourds > 3,5t' in x.keys() else np.nan)
        df_junction['Trottinettes + vélos'] = df_junction['mode_&_nb_usagers'].apply(lambda x : x['Trottinettes + vélos'] if 'Trottinettes + vélos' in x.keys() else np.nan)


In [21]:
df_junction

,label,latitude,longitude,t,mode,nb_usagers,mode_&_nb_usagers,2 roues motorisées,Trottinettes,"Véhicules légers < 3,5t",Vélos,Autobus et autocars,"Véhicules lourds > 3,5t",Trottinettes + vélos
0,[Paris] Rivoli x Nicolas Flamel,48.858273,2.349109,2020-05-22T22:00:00+00:00,"[Autobus et autocars, Trottinettes + vélos, Vé...","[5, 191, 85]","{'Autobus et autocars': 5, 'Trottinettes + vél...",NaN,NaN,85.0,NaN,5.0,NaN,191.0
1,[Paris] Rivoli x Nicolas Flamel,48.858273,2.349109,2020-05-22T23:00:00+00:00,"[Trottinettes + vélos, Véhicules légers < 3,5t]","[118, 86]","{'Trottinettes + vélos': 118, 'Véhicules léger...",NaN,NaN,86.0,NaN,NaN,NaN,118.0
2,[Paris] Rivoli x Nicolas Flamel,48.858273,2.349109,2020-05-23T00:00:00+00:00,"[Trottinettes + vélos, Véhicules légers < 3,5t]","[72, 42]","{'Trottinettes + vélos': 72, 'Véhicules légers...",NaN,NaN,42.0,NaN,NaN,NaN,72.0
3,[Paris] Rivoli x Nicolas Flamel,48.858273,2.349109,2020-05-23T01:00:00+00:00,"[Trottinettes + vélos, Véhicules légers < 3,5t]","[49, 46]","{'Trottinettes + vélos': 49, 'Véhicules légers...",NaN,NaN,46.0,NaN,NaN,NaN,49.0
4,[Paris] Rivoli x Nicolas Flamel,48.858273,2.349109,2020-05-23T02:00:00+00:00,"[Trottinettes + vélos, Véhicules lourds > 3,5t...","[31, 1, 25]","{'Trottinettes + vélos': 31, 'Véhicules lourds...",NaN,NaN,25.0,NaN,NaN,1.0,31.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25083,[Paris] Rivoli x Nicolas Flamel,48.858273,2.349109,2023-05-21T17:00:00+00:00,"[2 roues motorisées, Autobus et autocars, Trot...","[59, 10, 101, 1, 247, 809]","{'2 roues motorisées': 59, 'Autobus et autocar...",59.0,101.0,247.0,809.0,10.0,1.0,NaN
25084,[Paris] Rivoli x Nicolas Flamel,48.858273,2.349109,2023-05-21T18:00:00+00:00,"[2 roues motorisées, Autobus et autocars, Trot...","[47, 8, 99, 1, 204, 573]","{'2 roues motorisées': 47, 'Autobus et autocar...",47.0,99.0,204.0,573.0,8.0,1.0,NaN
25085,[Paris] Rivoli x Nicolas Flamel,48.858273,2.349109,2023-05-21T19:00:00+00:00,"[2 roues motorisées, Autobus et autocars, Trot...","[39, 8, 72, 3, 186, 364]","{'2 roues motorisées': 39, 'Autobus et autocar...",39.0,72.0,186.0,364.0,8.0,3.0,NaN
25086,[Paris] Rivoli x Nicolas Flamel,48.858273,2.349109,2023-05-21T20:00:00+00:00,"[2 roues motorisées, Autobus et autocars, Trot...","[33, 6, 50, 2, 183, 316]","{'2 roues motorisées': 33, 'Autobus et autocar...",33.0,50.0,183.0,316.0,6.0,2.0,NaN


In [22]:
df_junction.isnull().sum()

label                          0
latitude                       0
longitude                      0
t                              0
mode                           0
nb_usagers                     0
mode_&_nb_usagers              0
2 roues motorisées          1057
Trottinettes                2778
Véhicules légers < 3,5t       12
Vélos                       2631
Autobus et autocars         3242
Véhicules lourds > 3,5t     3738
Trottinettes + vélos       22472
dtype: int64

In [ ]:
df['2_wheelers'] = df['2 roues motorisées'] + df['Trottinettes'] + df['Trottinettes + vélos'] + df['Vélos']


df['4_wheelers'] = df['Véhicules légers < 3,5t'] + df['Véhicules lourds > 3,5t'] + df['Autobus et autocars']

In [4]:
dir_path = '../output/predictions/*/*.csv'
file_names = sorted(glob.glob(dir_path))

In [5]:
file_names

['../output/predictions/[Paris] Amsterdam x Clichy/df_pred_and_test.csv',
 '../output/predictions/[Paris] CF1 Rivoli x Sébastopol/df_pred_and_test.csv',
 '../output/predictions/[Paris] CF318 Rivoli x Lobau/df_pred_and_test.csv',
 '../output/predictions/[Paris] CF4 Poissonnière x Montmartre OUEST/df_pred_and_test.csv',
 '../output/predictions/[Paris] CF892 Rivoli x Bourdonnais/df_pred_and_test.csv',
 '../output/predictions/[Paris] Quai de Jemmapes/df_pred_and_test.csv',
 '../output/predictions/[Paris] Quai de Valmy/df_pred_and_test.csv',
 '../output/predictions/[Paris] Rivoli x Nicolas Flamel/df_pred_and_test.csv']

In [7]:
file = file_names[0]
df = pd.read_csv(file,index_col=0)
df['datetime'] = pd.to_datetime(df['datetime'])
df

,datetime,no_of_vehicles,type
0,2023-05-22 22:00:00+00:00,159.0,Prediction
1,2023-05-22 23:00:00+00:00,94.0,Prediction
2,2023-05-23 00:00:00+00:00,57.0,Prediction
3,2023-05-23 01:00:00+00:00,47.0,Prediction
4,2023-05-23 02:00:00+00:00,48.0,Prediction
...,...,...,...
166,2023-05-29 20:00:00+00:00,242.0,Actual
167,2023-05-29 21:00:00+00:00,209.0,Actual
168,2023-05-29 22:00:00+00:00,176.0,Actual
169,2023-05-29 23:00:00+00:00,188.0,Actual


In [10]:
#px.scatter(df, x="datetime", y="no_of_vehicles", color="type")
import plotly.express as px
fig = px.line(df, x="datetime", y="no_of_vehicles", color="type", markers=True,
               labels={
                     "no_of_vehicles": "No of vehicles passing each hour",
                     "datetime": "Datetime",
                 }, title=" test")
#fig.update_layout(xaxis=dict(tickmode="linear", tick0=0.0, dtick=1.0))
fig.update_traces(marker_size=10)
fig.update_xaxes(title_font=dict(size=16, family="Courier", color="black"))
fig.update_yaxes(title_font=dict(size=16, family="Courier", color="black"))
fig.show()

In [13]:
file.split('/')[-2]

'[Paris] Amsterdam x Clichy'